In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch
import pandas as pd
import numpy as np


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])


class ARIMA_LSTM:
    def __init__(self, ts, exog=None, lstm_params=None):
        """
        Initialize with time series data.

        Parameters
        ----------
        ts : pd.Series
            Time series data.
        exog : pd.DataFrame, optional
            Exogenous variables for ARIMA, by default None.
        lstm_params : dict, optional
            Dictionary with LSTM hyperparameters.
        """
        self.data = ts
        self.exog = exog
        self.arima_model = None
        self.lstm_model = None
        self.arima_residuals = None
        self.sequence_length = None

        if torch.backends.mps.is_available():
            self.device = torch.device("mps")
        elif torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

        # Default LSTM parameters
        self.lstm_params = lstm_params or {
            "input_size": 1,
            "hidden_size": 50,
            "num_layers": 1,
            "output_size": 1
        }

    def fit_arima(self, order=(1, 1, 1), seasonal=(0, 0, 0, 0), trend='c'):
        """Fits an ARIMA model and stores residuals."""
        model = sm.tsa.statespace.SARIMAX(self.data, self.exog, seasonal_order=seasonal, trend=trend, order=order)
        self.arima_model = model.fit()
        self.arima_residuals = self.arima_model.resid

    def prepare_lstm_data(self):
        """
        Prepares data for LSTM training from ARIMA residuals.

        Parameters
        ----------
        sequence_length : int, optional
            Length of input sequences, by default 10

        Returns
        -------
        DataLoader
            PyTorch DataLoader with formatted sequences and targets
        """
        if self.arima_residuals is None:
            raise ValueError("Fit ARIMA model first to generate residuals")

        # Convert residuals to numpy array
        residuals = self.arima_residuals.values.reshape(-1, 1)

        # Create sequences and targets
        X, y = [], []
        for i in range(len(residuals) - self.sequence_length):
            X.append(residuals[i:i + self.sequence_length])
            y.append(residuals[i + self.sequence_length])

        # Convert to PyTorch tensors
        X_tensor = torch.tensor(np.array(X), dtype=torch.float32)
        y_tensor = torch.tensor(np.array(y), dtype=torch.float32)

        # Create dataset and dataloader
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

        return dataloader

    def fit_lstm(self, epochs=20, lr=0.001, seq_length=10):
        """
        Fits an LSTM model on the residuals.

        Parameters
        ----------
        dataloader : DataLoader
            Dataloader for LSTM training.
        epochs : int, optional
            Number of training epochs, by default 20.
        lr : float, optional
            Learning rate, by default 0.001.
        """
        self.sequence_length = seq_length
        self.lstm_model = LSTMModel(**self.lstm_params).to(self.device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.lstm_model.parameters(), lr=lr)

        dataloader = self.prepare_lstm_data()
        self.lstm_model.train()
        for epoch in range(epochs):
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                optimizer.zero_grad()
                outputs = self.lstm_model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

    def predict(self, steps=1):
        """
        Forecasts mean (ARIMA) and residuals (LSTM).

        Parameters
        ----------
        steps : int, optional
            Number of steps to forecast, by default 1.

        Returns
        -------
        pd.Series
            ARIMA forecast.
        pd.Series
            LSTM residual forecast.
        """
        if self.arima_model is None or self.lstm_model is None:
            raise ValueError("Fit both ARIMA and LSTM before predicting.")

        arima_forecast = self.arima_model.forecast(steps=steps).values

        # Prepare residuals as LSTM input
        recent_residuals = self.arima_residuals[-self.sequence_length:].values.reshape(-1, 1)
        residuals_tensor = torch.tensor(recent_residuals, dtype=torch.float32).unsqueeze(
            0)  # Shape: (1, self.sequence_length, 1)

        self.lstm_model.eval()
        lstm_forecast = []

        # For multi-step forecasting
        with torch.no_grad():
            current_sequence = residuals_tensor.to(self.device)

            for _ in range(steps):
                # Get prediction for next step
                next_pred = self.lstm_model(current_sequence).cpu().numpy().flatten()[0]
                lstm_forecast.append(next_pred)

                # Update sequence for next prediction (rolling window approach)
                # Remove oldest value and append the prediction
                current_sequence = torch.cat([
                    current_sequence[:, 1:, :],
                    torch.tensor([[[next_pred]]], dtype=torch.float32).to(self.device)
                ], dim=1)

        forecast_index = pd.bdate_range(start=self.data.index[-1] + pd.tseries.offsets.BusinessDay(1), periods=steps)

        return pd.Series(arima_forecast, index=forecast_index), pd.Series(lstm_forecast, index=forecast_index)

    def plot_results(self, steps=10):
        """Plots historical data and recent predictions using `predict`."""
        if self.arima_model is None or self.lstm_model is None:
            raise ValueError("Fit both ARIMA and LSTM before plotting.")

        last_index = len(self.data) - steps
        arima_predicted = self.arima_model.predict(start=last_index, end=len(self.data) - 1)
        lstm_predicted = self.arima_residuals[-steps:]  # Approximation

        plt.figure(figsize=(10, 5))
        plt.plot(self.data, label='Actual Data', color='black')
        plt.plot(arima_predicted, label='ARIMA Predicted', linestyle='dashed', color='blue')
        plt.fill_between(arima_predicted.index,
                         arima_predicted - lstm_predicted,
                         arima_predicted + lstm_predicted,
                         color='gray', alpha=0.3, label='Residual Variability')

        plt.legend()
        plt.title(f"ARIMA+LSTM Predictions (Last {steps} Steps)")
        plt.show()

In [ ]:
dat = pd.read_csv("SP.csv", index_col="Date", parse_dates=True)[-200:]

In [ ]:
model = ARIMA_LSTM(dat['Adj Close'])
model.fit_arima()
model.fit_lstm(200, 0.01, 60)
arima, lstm = model.predict(60)

In [ ]:
model.plot_results(steps=60)